In [1]:
import pandas as pd
import numpy as np
import category_encoders as ce
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.pipeline import make_pipeline

In [3]:
df = pd.read_csv('data/master.csv', parse_dates=['visit_date'])

In [5]:
df.sort_values(by=['id', 'visit_date'], inplace=True)

In [4]:
# define some functions that we can reuse
def create_val_splits(df, val_units=15, return_val=False):
    """Function that will take in a dataset and split it up into training, validation, and test sets"""
    # split into training, validation, and test sets
    df = df.drop('visit_date', axis=1)
    train = df.groupby('id').apply(lambda x: x.iloc[:-val_units]).reset_index(drop=True)
    test  = df.groupby('id').apply(lambda x: x.iloc[-val_units:]).reset_index(drop=True)
    
    if return_val:
        val   = train.groupby('id').apply(lambda x: x.iloc[-val_units:]).reset_index(drop=True)
        train = train.groupby('id').apply(lambda x: x.iloc[:-val_units]).reset_index(drop=True)
        return train, val, test
    else:
        return train, test

In [9]:
df = df.fillna(0)

train, val, test = create_val_splits(df, return_val=True)

In [10]:
X_train, y_train = train.drop('visitors', axis=1), train['visitors']
X_val, y_val = val.drop('visitors', axis=1), val['visitors']
X_test, y_test = test.drop('visitors', axis=1), test['visitors']

In [7]:
df.isnull().sum()

id                       0
visit_date               0
visitors                 0
calendar_date            0
day_of_week              0
holiday                  0
genre                    0
area                     0
latitude                 0
longitude                0
reserve_visitors    143714
dtype: int64

In [8]:
df['reserve_visitors'].value_counts().sort_index()

1.0      1975
2.0     13587
3.0      5376
4.0      8042
5.0      3581
6.0      4627
7.0      6713
8.0      5639
9.0      3738
10.0     6703
11.0     2439
12.0     3677
13.0      744
14.0     1318
15.0     2259
16.0      704
17.0     1459
18.0     1047
20.0      722
21.0     2591
22.0      696
23.0     3564
24.0      613
25.0     3476
27.0      736
28.0      706
30.0     1297
31.0      738
32.0     1182
33.0      771
35.0     1358
36.0     2033
37.0      774
40.0      731
41.0      772
42.0      608
43.0     1372
44.0     1246
46.0     1393
47.0      771
48.0      603
51.0      659
54.0      767
58.0      574
59.0      672
60.0     1411
76.0      767
83.0      575
96.0      588
Name: reserve_visitors, dtype: int64

In [11]:
pipe = make_pipeline(ce.TargetEncoder(cols=['id']), ce.OrdinalEncoder(), GradientBoostingRegressor())

In [16]:
pipe[1].fit_transform(X_train)

/Users/umaprasad/opt/anaconda3/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


,id,calendar_date,day_of_week,holiday,genre,area,latitude,longitude,reserve_visitors
0,1,1,1,0,1,1,35.694003,139.753595,0.0
1,1,2,2,0,1,1,35.694003,139.753595,4.0
2,1,3,3,0,1,1,35.694003,139.753595,0.0
3,1,4,4,0,1,1,35.694003,139.753595,0.0
4,1,5,5,0,1,1,35.694003,139.753595,0.0
...,...,...,...,...,...,...,...,...,...
227243,828,407,7,0,11,87,35.708146,139.666288,44.0
227244,828,418,3,1,11,87,35.708146,139.666288,3.0
227245,828,419,4,0,11,87,35.708146,139.666288,7.0
227246,828,420,5,0,11,87,35.708146,139.666288,8.0
